In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string
from string import digits
nltk.download('punkt')
nltk.download('stopwords')
import xgboost as xgb
import numpy as np
import sklearn as sk

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


c:\users\radames\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv("../data/train.csv")
# 3 столбца - id, text, author
df_train.head(n=3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [3]:
remove_digits = str.maketrans('', '', digits)
def tokenize_stem(file_text):
    #firstly let's apply nltk tokenization
    file_text = file_text.translate(remove_digits)
    
    tokens = nltk.word_tokenize(file_text)

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #deleting stop_words
    stop_words = stopwords.words('english')
    tokens = [i for i in tokens if ( i not in stop_words )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return tokens

In [4]:
df_train['cleaned_text'] = df_train.text.apply(tokenize_stem)
df_train['cleaned_text_string'] = df_train.cleaned_text.apply(' '.join)
df_train.head(n=3)
eng_stopwords = set(stopwords.words("english"))

In [5]:
df_train['length']=df_train['cleaned_text_string'].apply(len)
df_train["num_words"] = df_train["text"].apply(lambda x: len(str(x).split()))
df_train["num_unique_words"] = df_train["text"].apply(lambda x: len(set(str(x).split())))
df_train["num_punctuations"] =df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_train["num_words_upper"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_train["num_words_title"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_train["mean_word_len"] = df_train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_train["num_stopwords"] = df_train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,3,4.658537,19
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,1,4.142857,8
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,1,4.583333,16


In [6]:
df_hpl=df_train[df_train['author']=='HPL']
df_hpl.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
count,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000
mean,93.395386,27.799645,24.437977,3.206921,0.500266,2.334694,4.625193,12.940728
std,51.075096,14.123252,11.053739,2.108637,0.852313,2.041579,0.554917,6.855502
min,7.000000,4.000000,3.000000,1.000000,0.000000,0.000000,2.222222,0.000000
25%,58.000000,18.000000,17.000000,2.000000,0.000000,1.000000,4.258065,8.000000
50%,85.000000,26.000000,23.000000,3.000000,0.000000,2.000000,4.600000,12.000000
75%,118.000000,35.000000,30.000000,4.000000,1.000000,3.000000,4.961538,16.500000
max,561.000000,147.000000,102.000000,28.000000,10.000000,39.000000,7.833333,62.000000


In [7]:
df_eap=df_train[df_train['author']=='EAP']
df_eap.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
count,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000
mean,81.543165,25.442405,21.894937,4.096329,0.553291,2.102405,4.644952,12.617722
std,60.100183,18.567706,13.727397,3.573788,0.892966,2.052241,0.631340,9.540922
min,5.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000
25%,40.000000,12.000000,12.000000,2.000000,0.000000,1.000000,4.250000,6.000000
50%,66.000000,21.000000,19.000000,3.000000,0.000000,1.000000,4.600000,10.000000
75%,106.000000,33.000000,29.000000,5.000000,1.000000,2.000000,5.000000,17.000000
max,925.000000,267.000000,155.000000,71.000000,15.000000,43.000000,11.000000,135.000000


In [8]:
df_mws=df_train[df_train['author']=='MWS']
df_mws.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
count,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000
mean,86.124586,27.417273,23.544672,3.833719,0.751489,2.124255,4.598182,13.742224
std,71.976281,23.134440,14.925835,2.840625,1.203636,1.759572,0.561558,12.080172
min,4.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.666667,0.000000
25%,48.000000,15.000000,14.000000,2.000000,0.000000,1.000000,4.250000,7.000000
50%,74.000000,23.000000,21.000000,3.000000,0.000000,2.000000,4.560791,12.000000
75%,108.000000,34.000000,30.000000,5.000000,1.000000,3.000000,4.907156,17.000000
max,2715.000000,861.000000,429.000000,59.000000,27.000000,46.000000,10.500000,435.000000


In [9]:
wordset=set()

In [10]:
#делаю сет со всеми словами
for i in df_train.index:
    wordset |= set(df_train['cleaned_text'][i])
wordlist=list(wordset)

In [11]:
#делаю фрейм со словами
df_word=pd.DataFrame(columns=["word", "mws", "eap", "hpl", "all"])
df_word["word"]=wordlist
df_word["mws"]=0
df_word["eap"]=0
df_word["hpl"]=0
df_word["all"]=0
df_word.head()

,word,mws,eap,hpl,all
0,silenc,0,0,0,0
1,enkindl,0,0,0,0
2,unti,0,0,0,0
3,outpour,0,0,0,0
4,drole,0,0,0,0


In [12]:
# как мы будем эту штуку правильнее делать (возможно это жуткий костыль), я хз
# сначала создаем словарь где ключ - уникальное слово, а значение - его порядковый номер
# затем создаем разреженную матрицу, которую заполняем в зависимости от порядковых номеров 
word_dict = {}



In [13]:
#делаю сет со всеми словами
# и сразу заготовку под шапку(потом увидишь зачем)
counter = 0
head = []

for wordlist in df_train['cleaned_text']:
    for word in wordlist:
        if word not in word_dict:
            head.append(word)
            word_dict[word] = counter
            counter += 1


In [14]:
# видоизменять колонки в pandas руками по одному значению в строке или столбце - очень плохая идея
# колонка это numpy.ndarray, а значит при каждой итерации она будет пересоздаваться
# что угробит производительность
# делаем значит так. считаем где сколько и где встречались отдельные слова, затем создаем строку за строкой для 
# каждого предложения

list_of_lists = []

for wordlist in df_train['cleaned_text']:
    row = [0 for i in range(len(word_dict))]
    for word in wordlist:
        row[word_dict[word]] += 1
    list_of_lists.append(row)



In [15]:
print(len(list_of_lists))

19579


In [16]:
# ... и для того чтобы посмотреть встречаемость того или иного слова по авторам добавим такую колонку

count_frame = pd.DataFrame(list_of_lists)
count_frame['author'] = df_train['author']



In [17]:
# ... и теперь нормальную шапку делаем

count_frame.columns = head + ['author']


In [18]:
print(count_frame.head())

   this  process  howev  afford  mean  ascertain  dimens  dungeon  i  might  \
0     1        1      1       1     1          1       1        1  2      1   
1     0        0      0       0     0          0       0        0  0      1   
2     0        0      0       0     0          0       0        0  0      0   
3     0        0      0       0     0          0       0        0  0      0   
4     0        0      0       0     0          0       0        0  0      0   

    ...    aegidus  burr  bentley  waltzer  binder  brusqueri  adriat  ancona  \
0   ...          0     0        0        0       0          0       0       0   
1   ...          0     0        0        0       0          0       0       0   
2   ...          0     0        0        0       0          0       0       0   
3   ...          0     0        0        0       0          0       0       0   
4   ...          0     0        0        0       0          0       0       0   

   agir  author  
0     0     EAP  
1 

In [19]:
# Пока объединим все, потом может быть будем использовать
col=list(count_frame.columns)
col[-1]='author_name'
count_frame.columns=col
pivot_col=pd.pivot_table(count_frame, aggfunc=np.sum, values=col, index=['author_name'])

In [20]:
#Убираем лишние слова, которые не учли раньше
col=list(pivot_col.columns)
col2=[string for string in col if (string[0]!='"' and string[0]!="'"
                                  and string[0]!='.' and string[0]!='`'
                                   and len(string)>3 and '.' not in string)]
col=[]
pivot_col=pivot_col[col2]
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
author_name,,,,,,,,,,,,,,,,,,,,,
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#Create pivot
pivot_col=pivot_col.append(pivot_col.sum(), ignore_index=True)
pivot_col.index=['EAP', 'HPL', 'MWS', 'SUMA']
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
SUMA,1,2,1,48,24,2,2,6,5,2,...,2,1,1,3,1,1,1,1,2,1


In [22]:
summa=[pivot_col.loc['EAP'].sum(), pivot_col.loc['HPL'].sum(), 
       pivot_col.loc['MWS'].sum(), pivot_col.loc['SUMA'].sum()]
pivot_col['summa']=summa
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος,summa
EAP,1,2,0,22,0,2,1,2,3,2,...,0,1,3,1,1,1,1,0,0,87765
HPL,0,0,0,17,24,0,1,3,0,0,...,1,0,0,0,0,0,0,2,1,74269
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,73160
SUMA,1,2,1,48,24,2,2,6,5,2,...,1,1,3,1,1,1,1,2,1,235194


In [23]:
# Create probability of author text knowing that a word was used
pivot_part=pivot_col
pivot_part.loc['EAP']=pivot_col.loc['EAP']/pivot_col.loc['SUMA']
pivot_part.loc['HPL']=pivot_col.loc['HPL']/pivot_col.loc['SUMA']
pivot_part.loc['MWS']=pivot_col.loc['MWS']/pivot_col.loc['SUMA']
pivot_part=pivot_part.loc[['EAP', 'HPL', 'MWS']]
# Delete unique words
pivot_part=pivot_part.loc[:, (pivot_part!=1).all(axis=0)]
pivot_part.head()

,abandon,abash,abat,abbey,abdic,aberr,abhor,abhorr,abil,abject,...,younger,youngest,your,youth,zeal,zenith,zest,zigzag,zone,summa
EAP,0.458333,0.5,0.333333,0.6,0.142857,0.166667,0.058824,0.111111,0.789474,0.333333,...,0.272727,0.2,0.534884,0.101562,0.117647,0.4,0.2,0.4,0.666667,0.373160
HPL,0.354167,0.5,0.500000,0.0,0.000000,0.666667,0.235294,0.555556,0.052632,0.000000,...,0.000000,0.4,0.069767,0.429688,0.470588,0.6,0.2,0.6,0.333333,0.315778
MWS,0.187500,0.0,0.166667,0.4,0.857143,0.166667,0.705882,0.333333,0.157895,0.666667,...,0.727273,0.4,0.395349,0.468750,0.411765,0.0,0.6,0.0,0.000000,0.311062


In [24]:
# It will be easier to work this way
eap_dict=pivot_part.loc['EAP'].to_dict()
hpl_dict=pivot_part.loc['HPL'].to_dict()
mws_dict=pivot_part.loc['MWS'].to_dict()
eap_dict['word']

0.44859813084112149

In [25]:
# Create author score 
def ind_val_eap(listn):
    quant=0
    for word in listn:
        try:
            quant+=eap_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_hpl(listn):
    quant=0
    for word in listn:
        try:
            quant+=hpl_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_mws(listn):
    quant=0
    for word in listn:
        try:
            quant+=mws_dict[word]
        except KeyError:
            quant+=0
    return quant

In [26]:
# Add index of author
df_train['mws_index']=df_train['cleaned_text'].apply(ind_val_mws)/df_train['length']
df_train['eap_index']=df_train['cleaned_text'].apply(ind_val_eap)/df_train['length']
df_train['hpl_index']=df_train['cleaned_text'].apply(ind_val_hpl)/df_train['length']
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,3,4.658537,19,0.035935,0.074388,0.034504
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,1,4.142857,8,0.035860,0.060980,0.034739
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,1,4.583333,16,0.026900,0.047832,0.037337


In [27]:
#Transform authors' names to numeric
df_train['author']=df_train['author'].astype('category')
df_train['author2']=df_train['author'].cat.codes
# Create different features 
df_train.head(n=3)
mid = df_train['author2']
df_train.drop(labels=['author2'], axis=1,inplace = True)
df_train.insert(0, 'author2', mid)
df_train.head()

,author2,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
0,0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,3,4.658537,19,0.035935,0.074388,0.034504
1,1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,1,4.142857,8,0.035860,0.060980,0.034739
2,0,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,1,4.583333,16,0.026900,0.047832,0.037337
3,2,id27763,How lovely is spring As we looked from Windsor...,MWS,"[how, love, spring, as, look, windsor, terrac,...",how love spring as look windsor terrac sixteen...,144,34,32,4,0,4,5.088235,13,0.071850,0.033438,0.033601
4,1,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[find, noth, els, even, gold, superintend, aba...",find noth els even gold superintend abandon at...,102,27,25,4,0,2,5.481481,11,0.036859,0.056661,0.043735


In [28]:
# Create data set
ds_train=df_train.values
X=ds_train[:, 6:]
Y=ds_train[:, 0]
seed=7
test_size=0.3
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X,Y, test_size=test_size, random_state=seed)

In [95]:
# fit model
xg_train=xgb.DMatrix(X_train, label=y_train)
xg_test=xgb.DMatrix(X_test, label=y_test)
xg_t=xgb.DMatrix(X, label=Y)
param={}
param['objective'] = 'multi:softmax'
param['eta'] = 0.3
param['max_depth'] = 3
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 3
param['eval_metric']= "mlogloss"
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 60
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))

[0]	train-mlogloss:0.88373	test-mlogloss:0.886188
[1]	train-mlogloss:0.754369	test-mlogloss:0.75956
[2]	train-mlogloss:0.666862	test-mlogloss:0.673645
[3]	train-mlogloss:0.606033	test-mlogloss:0.612389
[4]	train-mlogloss:0.560516	test-mlogloss:0.568099
[5]	train-mlogloss:0.528388	test-mlogloss:0.536639
[6]	train-mlogloss:0.503566	test-mlogloss:0.512748
[7]	train-mlogloss:0.484435	test-mlogloss:0.495375
[8]	train-mlogloss:0.469447	test-mlogloss:0.481325
[9]	train-mlogloss:0.455987	test-mlogloss:0.4696
[10]	train-mlogloss:0.445819	test-mlogloss:0.460479
[11]	train-mlogloss:0.437561	test-mlogloss:0.452738
[12]	train-mlogloss:0.430324	test-mlogloss:0.445819
[13]	train-mlogloss:0.42427	test-mlogloss:0.440757
[14]	train-mlogloss:0.418954	test-mlogloss:0.435865
[15]	train-mlogloss:0.414274	test-mlogloss:0.4322
[16]	train-mlogloss:0.409636	test-mlogloss:0.42779
[17]	train-mlogloss:0.40611	test-mlogloss:0.425315
[18]	train-mlogloss:0.403124	test-mlogloss:0.422984
[19]	train-mlogloss:0.400393	te

In [96]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_test).reshape(y_test.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.88373	test-mlogloss:0.886188
[1]	train-mlogloss:0.754369	test-mlogloss:0.75956
[2]	train-mlogloss:0.666862	test-mlogloss:0.673645
[3]	train-mlogloss:0.606033	test-mlogloss:0.612389
[4]	train-mlogloss:0.560516	test-mlogloss:0.568099
[5]	train-mlogloss:0.528388	test-mlogloss:0.536639
[6]	train-mlogloss:0.503566	test-mlogloss:0.512748
[7]	train-mlogloss:0.484435	test-mlogloss:0.495375
[8]	train-mlogloss:0.469447	test-mlogloss:0.481325
[9]	train-mlogloss:0.455987	test-mlogloss:0.4696
[10]	train-mlogloss:0.445819	test-mlogloss:0.460479
[11]	train-mlogloss:0.437561	test-mlogloss:0.452738
[12]	train-mlogloss:0.430324	test-mlogloss:0.445819
[13]	train-mlogloss:0.42427	test-mlogloss:0.440757
[14]	train-mlogloss:0.418954	test-mlogloss:0.435865
[15]	train-mlogloss:0.414274	test-mlogloss:0.4322
[16]	train-mlogloss:0.409636	test-mlogloss:0.42779
[17]	train-mlogloss:0.40611	test-mlogloss:0.425315
[18]	train-mlogloss:0.403124	test-mlogloss:0.422984
[19]	train-mlogloss:0.400393	te

In [97]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_t, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_t).reshape(Y.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != Y) / Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.884795	test-mlogloss:0.884144
[1]	train-mlogloss:0.75446	test-mlogloss:0.752468
[2]	train-mlogloss:0.669292	test-mlogloss:0.666306
[3]	train-mlogloss:0.607911	test-mlogloss:0.60434
[4]	train-mlogloss:0.563804	test-mlogloss:0.559629
[5]	train-mlogloss:0.531858	test-mlogloss:0.527268
[6]	train-mlogloss:0.507321	test-mlogloss:0.50255
[7]	train-mlogloss:0.488426	test-mlogloss:0.483624
[8]	train-mlogloss:0.472958	test-mlogloss:0.468459
[9]	train-mlogloss:0.460557	test-mlogloss:0.45577
[10]	train-mlogloss:0.450233	test-mlogloss:0.445697
[11]	train-mlogloss:0.44181	test-mlogloss:0.437132
[12]	train-mlogloss:0.434806	test-mlogloss:0.430056
[13]	train-mlogloss:0.428636	test-mlogloss:0.423997
[14]	train-mlogloss:0.423973	test-mlogloss:0.419289
[15]	train-mlogloss:0.419184	test-mlogloss:0.414543
[16]	train-mlogloss:0.41553	test-mlogloss:0.410961
[17]	train-mlogloss:0.411945	test-mlogloss:0.407194
[18]	train-mlogloss:0.408853	test-mlogloss:0.404279
[19]	train-mlogloss:0.40609	

In [98]:
df_test = pd.read_csv("../data/test.csv")
df_test.head(n=3)

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...


In [99]:
df_test['cleaned_text'] = df_test.text.apply(tokenize_stem)
df_test['cleaned_text_string'] = df_test.cleaned_text.apply(' '.join)
df_test.head(n=3)

,id,text,cleaned_text,cleaned_text_string
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, could, readili, fan, new...",if fire want fan could readili fan newspap gov...
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...


In [100]:
df_test['length']=df_test['cleaned_text_string'].apply(len)
df_test["num_words"] = df_test["text"].apply(lambda x: len(str(x).split()))
df_test["num_unique_words"] = df_test["text"].apply(lambda x: len(set(str(x).split())))
df_test["num_punctuations"] =df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test["num_words_upper"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_test["num_words_title"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_test["mean_word_len"] = df_test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test["num_stopwords"] = df_test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_test.head(n=3)

,id,text,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...,67,19,19,3,1,3,4.842105,9
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, could, readili, fan, new...",if fire want fan could readili fan newspap gov...,181,62,49,7,1,3,4.338710,33
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...,114,33,30,3,0,1,4.757576,15


In [101]:
df_test['mws_index']=df_test['cleaned_text'].apply(ind_val_mws)/df_test['length']
df_test['eap_index']=df_test['cleaned_text'].apply(ind_val_eap)/df_test['length']
df_test['hpl_index']=df_test['cleaned_text'].apply(ind_val_hpl)/df_test['length']
df_test.head(n=3)
df_test.head(n=3)

,id,text,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...,67,19,19,3,1,3,4.842105,9,0.071473,0.036315,0.026541
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, could, readili, fan, new...",if fire want fan could readili fan newspap gov...,181,62,49,7,1,3,4.338710,33,0.035932,0.062884,0.039306
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...,114,33,30,3,0,1,4.757576,15,0.027486,0.055142,0.057723


In [102]:
df_test[(df_test['mws_index']==df_test['hpl_index']) & (df_test['mws_index']==0)]

,id,text,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
1018,id12745,Who was the soundest of their logicians?,"[who, soundest, logician]",who soundest logician,21,7,7,1,0,1,4.857143,5,0.0,0.0,0.0
1983,id01671,"Oh Be men, or be more than men.","[oh, be, men, men]",oh be men men,13,8,8,2,0,2,3.000000,5,0.0,0.0,0.0
2288,id19562,Was there no injustice in this?,"[was, injustic]",was injustic,12,6,6,1,0,1,4.333333,4,0.0,0.0,0.0
3159,id06446,How about the somnambulism?,"[how, somnambul]",how somnambul,13,4,4,1,0,1,6.000000,3,0.0,0.0,0.0
3182,id12383,Can't be too snappish.,"[ca, n't, snappish]",ca n't snappish,15,4,4,2,0,0,4.750000,2,0.0,0.0,0.0
3404,id17653,"Yuh can't ask for nothin' we ain't got.""","[yuh, ca, n't, ask, nothin, ai, n't, got, '']",yuh ca n't ask nothin ai n't got '',35,8,8,5,0,1,4.125000,2,0.0,0.0,0.0
4160,id23738,An' the dogs was all barkin' an' whinin' awful.,"[an, dog, barkin, whinin, aw]",an dog barkin whinin aw,23,9,9,5,0,1,4.333333,3,0.0,0.0,0.0
5043,id09834,"We will say, then, that I am mad.","[we, say, i, mad]",we say i mad,12,8,8,3,1,2,3.250000,5,0.0,0.0,0.0
5085,id16162,"Matt he got to wonderin' too, an' so did Cap'n...","[matt, got, wonderin, cap'n, obe]",matt got wonderin cap'n obe,27,11,11,5,0,2,3.818182,4,0.0,0.0,0.0
5115,id07120,It was Monsieur Maillard himself.,"[it, monsieur, maillard]",it monsieur maillard,20,5,5,1,0,3,5.800000,2,0.0,0.0,0.0


In [103]:
ds_test=df_test.values

In [104]:
x_t=ds_test[:, 4:]
y_t=ds_test[:, 0]
xg_t=xgb.DMatrix(x_t)
pred_prob = bstp.predict(xg_t).reshape(y_t.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)

In [105]:
print(pred_prob)

[[ 0.00720587  0.00200887  0.99078524]
 [ 0.97838885  0.01382756  0.00778363]
 [ 0.03854186  0.96034449  0.00111369]
 ..., 
 [ 0.57887715  0.08871759  0.33240527]
 [ 0.02695208  0.01010656  0.96294141]
 [ 0.05490919  0.93719476  0.00789609]]


In [106]:
export=pd.DataFrame(pred_prob)
export.insert(loc=0, column='id', value=y_t)
export.columns=['id','EAP', 'HPL', 'MWS']
export.head()

,id,EAP,HPL,MWS
0,id02310,0.007206,0.002009,0.990785
1,id24541,0.978389,0.013828,0.007784
2,id00134,0.038542,0.960344,0.001114
3,id27757,0.858258,0.139627,0.002115
4,id04081,0.961801,0.008997,0.029201


In [107]:
len(export)

8392

In [108]:
export[export['id']=='id23301']

,id,EAP,HPL,MWS
6106,id23301,0.765783,0.195066,0.039151


In [109]:
export.to_csv(path_or_buf="../data/export.csv", index=False)